## Modeling ##

In [1]:

from sklearn.ensemble import GradientBoostingClassifier
import seaborn as sns
import shap
import pickle
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, 
    confusion_matrix,
    r2_score,
    mean_squared_error, 
    root_mean_squared_error,
    mean_absolute_error, 
    mean_absolute_percentage_error
)
from sklearn.inspection import PartialDependenceDisplay, permutation_importance, partial_dependence
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from scipy.spatial import KDTree

#from xgboost import XGBClassifier

from faiss_imputer import FaissImputer

from sklearn.impute import SimpleImputer

from sklearn.neural_network import MLPClassifier

This notebook is dedicated to the feature selection and statistical modeling of our trucking data.

In [4]:
df = pd.read_csv('../data/data_clean_bb.csv', low_memory=False,)
df
df.columns

Index(['Unnamed: 0', 'RecordID', 'ESS_Id', 'EventTimeStamp',
       'eventDescription', 'ecuModel', 'ecuMake', 'ecuSource', 'spn', 'fmi',
       'active', 'activeTransitionCount', 'EquipmentID', 'Latitude',
       'Longitude', 'LocationTimeStamp', 'FaultId', 'AcceleratorPedal',
       'BarometricPressure', 'CruiseControlActive', 'CruiseControlSetSpeed',
       'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad',
       'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm',
       'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature',
       'IgnStatus', 'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake',
       'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure',
       'next_derate_timestamp', 'time_until_detate', 'target'],
      dtype='object')

In [6]:
df['Throttle'] = df['Throttle'].str.replace(',', '.').astype(np.float64)

In [8]:
df['spn'] = df['spn'].astype(object)
df['fmi'] = df['fmi'].astype(object)
df = df.drop('Unnamed: 0', axis=1)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057049 entries, 0 to 1057048
Data columns (total 42 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   RecordID                   1057049 non-null  int64  
 1   ESS_Id                     1057049 non-null  int64  
 2   EventTimeStamp             1057049 non-null  object 
 3   eventDescription           1006152 non-null  object 
 4   ecuModel                   1001026 non-null  object 
 5   ecuMake                    1001026 non-null  object 
 6   ecuSource                  1057049 non-null  int64  
 7   spn                        1057049 non-null  object 
 8   fmi                        1057049 non-null  object 
 9   active                     1057049 non-null  bool   
 10  activeTransitionCount      1057049 non-null  int64  
 11  EquipmentID                1057049 non-null  object 
 12  Latitude                   1057049 non-null  float64
 13  Longitude   

In [12]:
test_date = '2019-01-01'
df_test = df.sort_values('EventTimeStamp').loc[df['EventTimeStamp'] > test_date]
df_train = df.sort_values('EventTimeStamp').loc[df['EventTimeStamp'] < test_date]


In [14]:
print(df_test.shape)
print(df_train.shape)

(111491, 42)
(945558, 42)


Scaling and encoding features for modeling

In [17]:
X_train = df_train.drop(columns = [
            'target',
            'LocationTimeStamp',
            'EventTimeStamp',
            'eventDescription',
            'Longitude',
            'Latitude',
            'ESS_Id',
            'RecordID',
            'ecuModel',
            'ecuMake',
            'SwitchedBatteryVoltage',
            'EquipmentID',
            'LampStatus',
            'CruiseControlSetSpeed',
            'EngineLoad',
            'TurboBoostPressure',
            'DistanceLtd',
            'FaultId', 
            'next_derate_timestamp', 
            'time_until_detate'
            ], axis=1)

y_train = df_train['target']

X_test = df_test.drop(columns = [
            'target',
            'LocationTimeStamp',
            'EventTimeStamp',
            'eventDescription',
            'Longitude',
            'Latitude',
            'ESS_Id',
            'RecordID',
            'ecuModel',
            'ecuMake',
            'SwitchedBatteryVoltage',
            'EquipmentID',
            'LampStatus',
            'CruiseControlSetSpeed',
            'EngineLoad',
            'TurboBoostPressure',
            'DistanceLtd',
            'FaultId', 
            'next_derate_timestamp', 
            'time_until_detate'
            ], axis=1)

y_test = df_test['target']

In [19]:
ohe_features = ['spn',
                'fmi',
                'ecuSource'
                ]
bool_features = ['CruiseControlActive',
                 'IgnStatus',
                 'ParkingBrake',
                 'active'
                ]
scale_features = ['AcceleratorPedal',
                  'BarometricPressure',
                  'EngineCoolantTemperature',
                  'EngineOilPressure',
                  'EngineOilTemperature',
                  'EngineRpm',
                  'FuelLevel',
                  'FuelLtd',
                  'FuelTemperature',
                  'IntakeManifoldTemperature',
                  'Speed',
                  'FuelRate',
                  'EngineTimeLtd',
                  'Throttle',
                  'activeTransitionCount'
                 ]


In [21]:
everything = list(set(ohe_features + bool_features + scale_features))
the_rest = X_train.columns.difference(everything)
the_rest

Index([], dtype='object')

In [ ]:
%%time



numerical_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    #('faiss', FaissImputer(n_neighbors=3, strategy='mean')),
    ('simple_imputer', SimpleImputer(strategy='mean'))
])
categorical_pipeline = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False, categories='auto')), 
    ('simple_imputer', SimpleImputer(strategy = 'most_frequent'))
])

bool_pipeline = Pipeline([
    ('simple_imputer', SimpleImputer(strategy = 'most_frequent'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, scale_features),
        ('cat', categorical_pipeline, ohe_features),
        ('bool', bool_pipeline, bool_features)
    ],
    remainder='drop'
)

pipe = Pipeline(steps=[('transformer', preprocessor)])

pipe.fit(X_train)

X_train_transformed = pipe.transform(X_train)
X_test_transformed = pipe.transform(X_test)

In [ ]:
smote = SMOTE()
X_trained_balanced, y_trained_balanced = smote.fit_resample(X_train_transformed, y_train)

In [ ]:
#filename = 'pipe_transformed.pkl'

#pickle_list = [pipe, X_train_transformed, X_test_transformed]

#with open(filename, 'wb') as file:
    #pickle.dump(pickle_list, file)

In [ ]:
#with open(filename, 'rb') as file:
    #pipe, X_train_transformed, X_test_transformed = pickle.load(file)

In [ ]:
X_trained_balanced

In [ ]:
%%time

feature_names = X_trained_balanced.columns.tolist()
forest = RandomForestClassifier(random_state=42)
forest.fit(X_train_transformed, y_train)

In [ ]:
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
forest_importances = pd.Series(importances, index=feature_names)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
feature_names

%%time

gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=4)

gbc.fit(X_trained_balanced, y_trained_balanced)

y_pred = gbc.predict(X_test_transformed)

In [ ]:
gbc.feature_importances_


In [ ]:
from sklearn.inspection import PartialDependenceDisplay

In [ ]:
variable_names = X_trained_balanced.columns.tolist()
pd.DataFrame({
    'variable': variable_names,
    'gbc_coeff': gbc.coef_
})

In [ ]:
pd.DataFrame({
    'variable': variable_names,
    'importance': permutation_importance(gbc, X_test_transformed, y_test, random_state = 321)['importances_mean']
}).sort_values('importance', ascending = False)

In [ ]:
features = X_test_transformed.columns.tolist()

pdp = partial_dependence(gbc, X_test_transformed, features = features)

for i in features:
    plt.plot(pdp['grid_values'][0], pdp['average'][i])
plt.legend();

In [ ]:
explainer = shap.TreeExplainer(gbc)
explanation = explainer(X_test)


In [ ]:
shap.plots.beeswarm(explanation)

In [ ]:
shap.plots.bar(explanation)

In [ ]:
pd.DataFrame(X_test).info()

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred, zero_division = 0))

In [ ]:
from sklearn.linear_model import RidgeClassifier
RC = RidgeClassifier(alpha=1.0)
RC.fit(X_train_transformed, y_train)
RC_y_pred = RC.predict(X_test_transformed)
#print(f'Accuracy: {accuracy_score(y_test, RC_y_pred)}')
#print(f'MCC: {matthews_corrcoef(y_test, RC_y_pred)}')
print(confusion_matrix(y_test, RC_y_pred))
print(classification_report(y_test, RC_y_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression
LG = LogisticRegression(penalty='l1', solver='liblinear')
LG.fit(X_train_transformed, y_train)
LG_y_pred = LG.predict(X_test_transformed)
#print(f'Accuracy: {accuracy_score(y_test, RC_y_pred)}')
#print(f'MCC: {matthews_corrcoef(y_test, RC_y_pred)}')
print(confusion_matrix(y_test, RC_y_pred))
print(classification_report(y_test, RC_y_pred))

In [ ]:
%%time

mlp = MLPClassifier(
            hidden_layer_sizes = (17, 15, 13, 11),
            activation = 'relu',
            solver = 'adam',            
            max_iter = 100000, 
            alpha = 0.5,
            learning_rate = 'adaptive'
        ).fit(X_train_transformed, y_train)

y_pred_mlp = mlp.predict(X_test_transformed)

In [ ]:
confusion_matrix(y_test, y_pred_mlp)

In [ ]:
print(classification_report(y_test, y_pred_mlp, zero_division = 0))

In [ ]:
corr_matrix = X_train[scale_features].corr().melt(ignore_index=False)

In [ ]:
corr_matrix[corr_matrix['value'] != 1].sort_values(by = 'value', ascending=False).head(12)

In [ ]:
sns.heatmap(corr_matrix, annot=True, cmap='YlGnBu', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()